<a href="https://colab.research.google.com/github/potis/AISummit/blob/main/cifar10_baseline_additional_material.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch OOD


## https://pytorch-ood.readthedocs.io/


### Maximum Softmax Probability

Implements the Maximum Softmax Probability (MSP) Thresholding baseline for OOD detection.
Optionally, implements temperature scaling, which divides the logits by a constant temperature **T** before calculating the softmax.

https://arxiv.org/abs/1610.02136

### Max Logit Method

Implements the Max Logit Method for OOD Detection as proposed in Scaling Out-of-Distribution Detection for Real-World Settings.


https://arxiv.org/abs/1911.11132



### OpenMax
Implementation of the OpenMax Layer as proposed in the paper Towards Open Set Deep Networks.

The methods determines a center for each class in the logits space of a model, and then creates a statistical model of the distances of correct classified inputs. It uses extreme value theory to detect outliers by fitting a weibull function to the tail of the distance distribution.

We use the activation of the unknown class as outlier score.

https://arxiv.org/abs/1511.06233

### ODIN Preprocessing

Implements ODIN from the paper Enhancing The Reliability of Out-of-distribution Image Detection in Neural Networks.

ODIN is a preprocessing method for inputs that aims to increase the discriminability of the softmax outputs for IN and OOD data.


https://arxiv.org/abs/1706.02690


### EnergyBased
Implements the Energy Score of Energy-based Out-of-distribution Detection.

This methods calculates the negative energy for a vector of logits. This value can be used as outlier score.


https://proceedings.neurips.cc/paper/2020/file/f5496252609c43eb8a3d147ab9b9c006-Paper.pdf


### Mahalanobis Method

Implements the Mahalanobis Method from the paper A Simple Unified Framework for Detecting Out-of-Distribution Samples and Adversarial Attacks.


### Monte Carlo Dropout

From the paper Dropout as a Bayesian Approximation: Representing Model Uncertainty in Deep Learning. Forward-propagates the input through the model several times with activated dropout and averages the results.


http://proceedings.mlr.press/v48/gal16.pdf


### Virtual Logit Matching

Implements Virtual Logit Matching (ViM) from the paper ViM: Out-Of-Distribution with Virtual-logit Matching.

https://arxiv.org/abs/2203.10807


### KL-Matching
Implements KL-Matching from the paper Scaling Out-of-Distribution Detection for Real-World Settings.

https://arxiv.org/abs/1911.11132

### Entropy-based
This methods calculates the entropy based on the logits of a classifier. Higher entropy means more uniformly distributed posteriors, indicating larger uncertainty



In [3]:
!pip install pytorch_ood

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 15.8 MB/s eta 0:00:00


In [4]:
import pandas as pd  # additional dependency, used here for convenience
import torch
import torchvision.transforms as tvt
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10

from pytorch_ood.dataset.img import (
    LSUNCrop,
    LSUNResize,
    Textures,
    TinyImageNetCrop,
    TinyImageNetResize,
)
from pytorch_ood.detector import (
    ODIN,
    EnergyBased,
    Entropy,
    KLMatching,
    Mahalanobis,
    MaxLogit,
    MaxSoftmax,
    ViM,
)
from pytorch_ood.model import WideResNet
from pytorch_ood.utils import OODMetrics, ToRGB, ToUnknown

device = "cuda:0"

torch.manual_seed(123)

Setup preprocessing



In [5]:
mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]
trans = tvt.Compose(
    [tvt.Resize(size=(32, 32)), ToRGB(), tvt.ToTensor(), tvt.Normalize(std=std, mean=mean)]
)

Setup datasets



In [6]:
dataset_in_test = CIFAR10(root="data", train=False, transform=trans, download=True)

# create all OOD datasets
ood_datasets = [Textures, LSUNCrop]
datasets = {}
for ood_dataset in ood_datasets:
    dataset_out_test = ood_dataset(
        root="data", transform=trans, target_transform=ToUnknown(), download=True
    )
    test_loader = DataLoader(dataset_in_test + dataset_out_test, batch_size=256)
    datasets[ood_dataset.__name__] = test_loader

100%|██████████| 170498071/170498071 [00:02<00:00, 74446703.62it/s]


Extracting data/cifar-10-python.tar.gz to data


100%|██████████| 625239812/625239812 [00:19<00:00, 31986946.13it/s]


Extracting data/textures-r1_0_1.tar.gz to data


100%|██████████| 26501958/26501958 [00:00<00:00, 104334619.04it/s]


...

Extracting data/Imagenet_resize.tar.gz to data


100%|██████████| 17309383/17309383 [00:00<00:00, 22960213.29it/s]


Extracting data/LSUN.tar.gz to data


100%|██████████| 4688973/4688973 [00:00<00:00, 28546224.13it/s]


Extracting data/LSUN_resize.tar.gz to data


**Stage 1**: Create DNN with pre-trained weights from the Hendrycks baseline paper



In [7]:
print("STAGE 1: Creating a Model")
model = WideResNet(num_classes=10, pretrained="cifar10-pt").eval().to(device)

STAGE 1: Creating a Model


Downloading: "https://github.com/wetliu/energy_ood/raw/master/CIFAR/snapshots/pretrained/cifar10_wrn_pretrained_epoch_99.pt" to /root/.cache/torch/hub/checkpoints/wrn-cifar10-pt.pt
100%|██████████| 8.62M/8.62M [00:00<00:00, 63.3MB/s]


**Stage 2**: Create OOD detector



In [8]:
print("STAGE 2: Creating OOD Detectors")
detectors = {}
detectors["Entropy"] = Entropy(model)
detectors["ViM"] = ViM(model.features, d=64, w=model.fc.weight, b=model.fc.bias)
detectors["Mahalanobis"] = Mahalanobis(model.features, norm_std=std, eps=0.002)
detectors["KLMatching"] = KLMatching(model)
detectors["MaxSoftmax"] = MaxSoftmax(model)
detectors["EnergyBased"] = EnergyBased(model)
detectors["MaxLogit"] = MaxLogit(model)
detectors["ODIN"] = ODIN(model, norm_std=std, eps=0.002)

# fit detectors to training data (some require this, some do not)
print(f"> Fitting {len(detectors)} detectors")
loader_in_train = DataLoader(CIFAR10(root="data", train=True, transform=trans), batch_size=256)
for name, detector in detectors.items():
    print(f"--> Fitting {name}")
    detector.fit(loader_in_train, device=device)

STAGE 2: Creating OOD Detectors
> Fitting 8 detectors
--> Fitting Entropy
--> Fitting ViM
--> Fitting Mahalanobis
--> Fitting KLMatching
--> Fitting MaxSoftmax
--> Fitting EnergyBased
--> Fitting MaxLogit
--> Fitting ODIN


**Stage 3**: Evaluate Detectors



In [9]:
print(f"STAGE 3: Evaluating {len(detectors)} detectors on {len(datasets)} datasets.")
results = []

with torch.no_grad():
    for detector_name, detector in detectors.items():
        print(f"> Evaluating {detector_name}")
        for dataset_name, loader in datasets.items():
            print(f"--> {dataset_name}")
            metrics = OODMetrics()
            for x, y in loader:
                metrics.update(detector(x.to(device)), y.to(device))

            r = {"Detector": detector_name, "Dataset": dataset_name}
            r.update(metrics.compute())
            results.append(r)

# calculate mean scores over all datasets, use percent
df = pd.DataFrame(results)
mean_scores = df.groupby("Detector").mean() * 100
print(mean_scores.sort_values("AUROC").to_csv(float_format="%.2f"))

STAGE 3: Evaluating 8 detectors on 5 datasets.
> Evaluating Entropy
--> Textures
--> TinyImageNetCrop
--> TinyImageNetResize
--> LSUNCrop
--> LSUNResize
> Evaluating ViM
--> Textures
--> TinyImageNetCrop
--> TinyImageNetResize
--> LSUNCrop
--> LSUNResize
> Evaluating Mahalanobis
--> Textures
--> TinyImageNetCrop
--> TinyImageNetResize
--> LSUNCrop
--> LSUNResize
> Evaluating KLMatching
--> Textures
--> TinyImageNetCrop
--> TinyImageNetResize
--> LSUNCrop
--> LSUNResize
> Evaluating MaxSoftmax
--> Textures
--> TinyImageNetCrop
--> TinyImageNetResize
--> LSUNCrop
--> LSUNResize
> Evaluating EnergyBased
--> Textures
--> TinyImageNetCrop
--> TinyImageNetResize
--> LSUNCrop
--> LSUNResize
> Evaluating MaxLogit
--> Textures
--> TinyImageNetCrop
--> TinyImageNetResize
--> LSUNCrop
--> LSUNResize
> Evaluating ODIN
--> Textures
--> TinyImageNetCrop
--> TinyImageNetResize
--> LSUNCrop
--> LSUNResize
Detector,AUROC,AUPR-IN,AUPR-OUT,FPR95TPR
KLMatching,88.73,86.95,85.10,58.73
MaxSoftmax,91.85,88.5

<ipython-input-9-875d67aec215>:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mean_scores = df.groupby("Detector").mean() * 100


In [18]:
print(f"STAGE 3: Evaluating {len(detectors)} detectors on {len(datasets)} datasets.")
results = []

with torch.no_grad():
    for detector_name, detector in detectors.items():
        print(f"> Evaluating {detector_name}")
        for dataset_name, loader in datasets.items():
            print(f"--> {dataset_name}")
            metrics = OODMetrics()
            x,y=next(iter(loader))
            print(dataset_name, dataset_name)
            prediction=detector(x[0:1].to(device))
            print(prediction)

#             r = {"Detector": detector_name, "Dataset": dataset_name}
#             r.update(metrics.compute())
#             results.append(r)

# # calculate mean scores over all datasets, use percent
# df = pd.DataFrame(results)
# mean_scores = df.groupby("Detector").mean() * 100
# print(mean_scores.sort_values("AUROC").to_csv(float_format="%.2f"))

STAGE 3: Evaluating 8 detectors on 5 datasets.
> Evaluating Entropy
--> Textures
Textures Textures
tensor([3.8212e-05], device='cuda:0')
--> TinyImageNetCrop
TinyImageNetCrop TinyImageNetCrop
tensor([3.8212e-05], device='cuda:0')
--> TinyImageNetResize
TinyImageNetResize TinyImageNetResize
tensor([3.8212e-05], device='cuda:0')
--> LSUNCrop
LSUNCrop LSUNCrop
tensor([3.8212e-05], device='cuda:0')
--> LSUNResize
LSUNResize LSUNResize
tensor([3.8212e-05], device='cuda:0')
> Evaluating ViM
--> Textures
Textures Textures
tensor([-4.0757])
--> TinyImageNetCrop
TinyImageNetCrop TinyImageNetCrop
tensor([-4.0757])
--> TinyImageNetResize
TinyImageNetResize TinyImageNetResize
tensor([-4.0757])
--> LSUNCrop
LSUNCrop LSUNCrop
tensor([-4.0757])
--> LSUNResize
LSUNResize LSUNResize
tensor([-4.0757])
> Evaluating Mahalanobis
--> Textures
Textures Textures
tensor([0.0012], device='cuda:0')
--> TinyImageNetCrop
TinyImageNetCrop TinyImageNetCrop
tensor([0.0012], device='cuda:0')
--> TinyImageNetResize
Tin